<a href="https://colab.research.google.com/github/MaxMatteucci/EventRecommender/blob/main/ProfBuilderAndRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import io
import os

# Try to load directly if file is already in /content
path = "/content/tm_indiana_chicago_events_with_capacity.csv"

if os.path.exists(path):
    print("Loading file from /content...")
    df_main = pd.read_csv(path)
else:
    print("File not found in /content — upload it now.")
    uploaded = files.upload()  # opens file picker

    # Get the uploaded filename (first key)
    filename = list(uploaded.keys())[0]

    df_main = pd.read_csv(io.BytesIO(uploaded[filename]))

df_main.head()


Loading file from /content...


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,event_id,name,segment,genre,subgenre,venue_name,city,state,country,datetime_utc,url,event_date,event_time,Y,X,event_lat,event_lon,capacity
0,vv17fZbsGkReyoNe,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-06 00:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-06,NaN,39.7684,-86.1581,39.7684,-86.1581,2400.0
1,vv17fZbsGkRezsxY,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-05 00:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-05,NaN,39.7684,-86.1581,39.7684,-86.1581,2400.0
2,vv17fZbsGkReBoN1,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-06 19:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-06,19:00:00,39.7684,-86.1581,39.7684,-86.1581,2400.0
3,vv17fZbsGkReysN6,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-07 01:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-07,01:00:00,39.7684,-86.1581,39.7684,-86.1581,2400.0
4,vv17fZbsGkReBsN2,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-08 00:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-08,NaN,39.7684,-86.1581,39.7684,-86.1581,2400.0


In [ ]:
import pandas as pd

df_main= pd.read_csv("/content/tm_indiana_chicago_events_with_capacity.csv")
df_main.head()


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,event_id,name,segment,genre,subgenre,venue_name,city,state,country,datetime_utc,url,event_date,event_time,Y,X,event_lat,event_lon,capacity
0,vv17fZbsGkReyoNe,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-06 00:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-06,NaN,39.7684,-86.1581,39.7684,-86.1581,2400.0
1,vv17fZbsGkRezsxY,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-05 00:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-05,NaN,39.7684,-86.1581,39.7684,-86.1581,2400.0
2,vv17fZbsGkReBoN1,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-06 19:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-06,19:00:00,39.7684,-86.1581,39.7684,-86.1581,2400.0
3,vv17fZbsGkReysN6,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-07 01:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-07,01:00:00,39.7684,-86.1581,39.7684,-86.1581,2400.0
4,vv17fZbsGkReBsN2,Hamilton (Touring),Arts & Theatre,Theatre,Musical,Old National Centre,Indianapolis,IN,United States Of America,2025-12-08 00:00:00+00:00,https://www.ticketmaster.com/hamilton-touring-...,2025-12-08,NaN,39.7684,-86.1581,39.7684,-86.1581,2400.0


In [ ]:
import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime, timedelta

# ------------------------------------------------------
# 0. df_main MUST ALREADY EXIST (with city, genre, subgenre, segment, event_date, venue, capacity)
# ------------------------------------------------------

df_main = df_main.copy()

# Fill capacity if missing
df_main["capacity"] = df_main["capacity"].fillna(0)

# ------------------------------------------------------
# 1. CITY COORDINATES
# ------------------------------------------------------

city_coords = {
    "Indianapolis": (39.7684, -86.1581),
    "Fort Wayne": (41.0793, -85.1394),
    "Evansville": (37.9716, -87.5711),
    "South Bend": (41.6764, -86.2520),
    "Bloomington": (39.1653, -86.5264),
    "Lafayette": (40.4167, -86.8753),
    "West Lafayette": (40.4259, -86.9081),
    "Chicago": (41.8781, -87.6298),
    "Springfield": (39.7817, -89.6501),
    "Rockford": (42.2711, -89.0937),
    "Peoria": (40.6936, -89.5890),
    "Champaign": (40.1164, -88.2434),
    "Naperville": (41.7508, -88.1535),
}

cities = list(city_coords.keys())


# ------------------------------------------------------
# 2. SEGMENT → GENRE / SUBGENRE MAP
# ------------------------------------------------------

segment_map = {}
for seg in ["Arts & Theatre", "Music", "Sports"]:
    seg_df = df_main[df_main["segment"] == seg]
    segment_map[seg] = {
        "genres": sorted(seg_df["genre"].dropna().unique()),
        "subgenres": sorted(seg_df["subgenre"].dropna().unique()),
    }

def allowed_choices(selected_segments):
    genres, subgenres = set(), set()
    for seg in selected_segments:
        genres.update(segment_map.get(seg, {}).get("genres", []))
        subgenres.update(segment_map.get(seg, {}).get("subgenres", []))
    return sorted(genres), sorted(subgenres)


# ------------------------------------------------------
# 3. USER SURVEY PIPELINE
# ------------------------------------------------------

user_profile = {}

print("\nSelect your home city:")
for i, c in enumerate(cities):
    print(f"{i+1}. {c}")

try:
    home_idx = int(input("\nEnter your home city number: ")) - 1
    home_city = cities[home_idx]
except:
    home_city = "Indianapolis"

user_profile["home_lat"], user_profile["home_lon"] = city_coords[home_city]

segments_available = ["Arts & Theatre", "Music", "Sports"]

print("\nWhich event TYPES do you enjoy? (e.g., 1,3)")
for i, seg in enumerate(segments_available):
    print(f"{i+1}. {seg}")

try:
    seg_choices = input("\nYour choices: ")
    chosen_segments = [segments_available[int(x)-1] for x in seg_choices.split(",")]
except:
    chosen_segments = ["Music"]

user_profile["segments"] = chosen_segments

# Allowed genres/subgenres
genres_allowed, subgenres_allowed = allowed_choices(chosen_segments)

print("\nRank genres (ex: 4,1,2). Leave blank to skip:")
for i, g in enumerate(genres_allowed):
    print(f"{i+1}. {g}")

raw_g = input("\nGenres: ")
ranked_genres = []
if raw_g.strip():
    try:
        ranked_genres = [genres_allowed[int(x)-1] for x in raw_g.split(",")]
    except:
        ranked_genres = []
user_profile["ranked_genres"] = ranked_genres

print("\nRank subgenres (optional):")
for i, sg in enumerate(subgenres_allowed):
    print(f"{i+1}. {sg}")

raw_sg = input("\nSubgenres: ")
ranked_subgenres = []
if raw_sg.strip():
    try:
        ranked_subgenres = [subgenres_allowed[int(x)-1] for x in raw_sg.split(",")]
    except:
        ranked_subgenres = []
user_profile["ranked_subgenres"] = ranked_subgenres


print("\nDate preference:")
print("1. Weekends only")
print("2. Weekdays only")
print("3. Any time")
print("4. Next 30 days")
print("5. Next 60 days")
print("6. Next 90 days")
print("7. Custom date range")

pref = input("\nChoose one: ")
user_profile["date_preference"] = pref

if pref == "7":
    start = input("Start date YYYY-MM-DD: ")
    end = input("End date YYYY-MM-DD: ")
    user_profile["custom_range"] = (start, end)

print("\n🎉 USER PROFILE CREATED:")
print(user_profile)


# ------------------------------------------------------
# 4. SCORING FUNCTIONS
# ------------------------------------------------------

df_main["event_lat"] = df_main["city"].map(lambda x: city_coords.get(x, (None,None))[0])
df_main["event_lon"] = df_main["city"].map(lambda x: city_coords.get(x, (None,None))[1])

def haversine(lat1, lon1, lat2, lon2):
    if pd.isna(lat2) or pd.isna(lon2):
        return 9999
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    return 2 * R * atan2(
        sqrt(sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2)*sin((lon2-lon1)/2)**2),
        sqrt(1 - (sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2)*sin((lon2-lon1)/2)**2))
    )

def compute_distance_score(row, ulat, ulon):
    d = haversine(ulat, ulon, row["event_lat"], row["event_lon"])
    return max(0, min(1, 1 - d/200))


def compute_genre_score(g, ranked):
    if not ranked or pd.isna(g): return 0
    if g not in ranked: return 0
    return 1 / (ranked.index(g) + 1)


def compute_subgenre_score(sg, ranked):
    if not ranked or pd.isna(sg): return 0
    if sg not in ranked: return 0
    return 1 / (ranked.index(sg) + 1)


def compute_segment_score(seg, selected_segments):
    if pd.isna(seg): return 0
    return 1 if seg in selected_segments else 0


def compute_date_score(date, profile):
    if pd.isna(date): return 0
    date = pd.to_datetime(date).date()
    today = datetime.now().date()
    pref = profile["date_preference"]

    if pref == "1": return 1 if date.weekday() >= 5 else 0
    if pref == "2": return 1 if date.weekday() < 5 else 0
    if pref == "3": return 1
    if pref in ("4","5","6"):
        limit = {"4":30, "5":60, "6":90}[pref]
        return 1 if date <= today + timedelta(days=limit) else 0
    if pref == "7":
        start, end = profile["custom_range"]
        start = pd.to_datetime(start).date()
        end = pd.to_datetime(end).date()
        return 1 if start <= date <= end else 0
    return 1


# ---- Capacity bump (NEW) ----
def compute_capacity_score(cap):
    if pd.isna(cap) or cap <= 0:
        return 0

    cap_log = np.log10(cap)
    score = (cap_log - 2.0) / (5.0 - 2.0)
    score = max(0, min(score, 1))
    return score * 0.5   # max bump = 0.5


# ------------------------------------------------------
# 5. RECOMMENDER WITH STRICT EXACT MATCHING
# ------------------------------------------------------

def build_recommendations(df, profile, top_n=10):
    df = df.copy()
    ulat, ulon = profile["home_lat"], profile["home_lon"]

    df["distance_score"] = df.apply(lambda r: compute_distance_score(r, ulat, ulon), axis=1)
    df["genre_score"] = df["genre"].apply(lambda g: compute_genre_score(g, profile["ranked_genres"]))
    df["subgenre_score"] = df["subgenre"].apply(lambda sg: compute_subgenre_score(sg, profile["ranked_subgenres"]))
    df["segment_score"] = df["segment"].apply(lambda s: compute_segment_score(s, profile["segments"]))
    df["date_score"] = df["event_date"].apply(lambda d: compute_date_score(d, profile))
    df["capacity_score"] = df["capacity"].apply(compute_capacity_score)

    # Updated weights with capacity
    df["final_score"] = (
        0.48 * df["segment_score"] +
        0.20 * df["genre_score"] +
        0.10 * df["subgenre_score"] +
        0.12 * df["distance_score"] +
        0.05 * df["date_score"] +
        0.05 * df["capacity_score"]
    )

    # STRICT EXACT MATCH:
    exact_mask = (
        (df["segment_score"] == 1) &
        (df["genre_score"] > 0) &
        (df["subgenre_score"] > 0) &
        (df["date_score"] == 1)
    )

    exact = df[exact_mask].sort_values("final_score", ascending=False).head(top_n)
    similar = df[~exact_mask].sort_values("final_score", ascending=False).head(top_n)

    return exact, similar


# ------------------------------------------------------
# 6. RUN ENGINE
# ------------------------------------------------------

exact_matches, similar_matches = build_recommendations(df_main, user_profile, top_n=10)

print("\n🎯 EXACT MATCHES (Top 10):")
display(exact_matches)


print("\n✨ SIMILAR MATCHES (Top 10):")
display(similar_matches)




Select your home city:
1. Indianapolis
2. Fort Wayne
3. Evansville
4. South Bend
5. Bloomington
6. Lafayette
7. West Lafayette
8. Chicago
9. Springfield
10. Rockford
11. Peoria
12. Champaign
13. Naperville

Enter your home city number: 7

Which event TYPES do you enjoy? (e.g., 1,3)
1. Arts & Theatre
2. Music
3. Sports

Your choices: 3

Rank genres (ex: 4,1,2). Leave blank to skip:
1. Baseball
2. Basketball
3. Boxing
4. Diving
5. Football
6. Hockey
7. Miscellaneous
8. Motorsports/Racing
9. Other
10. Rodeo
11. Track & Field
12. Wrestling

Genres: 2

Rank subgenres (optional):
1. AHL
2. Boxing
3. Bullriding
4. College
5. Diving
6. High School
7. Ice Hockey
8. MLB
9. Minor League
10. Miscellaneous
11. Motorsports/Racing
12. NBA
13. NFL
14. NHL
15. Other
16. Rodeo
17. Track & Field
18. Wrestling

Subgenres: 4,14

Date preference:
1. Weekends only
2. Weekdays only
3. Any time
4. Next 30 days
5. Next 60 days
6. Next 90 days
7. Custom date range

Choose one: 

🎉 USER PROFILE CREATED:
{'home_l

,event_id,name,segment,genre,subgenre,venue_name,city,state,country,datetime_utc,...,event_lat,event_lon,capacity,distance_score,genre_score,subgenre_score,segment_score,date_score,capacity_score,final_score
495,Z7r9jZ1A7qv_k,Purdue Boilermakers Mens Basketball vs. Michig...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2026-02-17 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
496,Z7r9jZ1A7qNqd,Purdue Boilermakers Womens Basketball vs. Iowa...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2026-02-20 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
497,Z7r9jZ1A7qv_6,Purdue Boilermakers Mens Basketball vs. Indian...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2026-02-20 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
498,Z7r9jZ1A7qNqv,Purdue Boilermakers Womens Basketball vs. Oreg...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2026-02-26 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
499,Z7r9jZ1A7qv_F,Purdue Boilermakers Mens Basketball vs. Michig...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2026-02-26 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
478,Z7r9jZ1A7j7af,Purdue Boilermakers Womens Basketball vs. Jack...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2025-12-21 17:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
479,Z7r9jZ1A7qNqA,Purdue Boilermakers Womens Basketball vs. Illi...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2025-12-28 17:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
480,Z7r9jZ1A7Q__N,Purdue Boilermakers Mens Basketball vs. Kent S...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2025-12-29 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
481,Z7r9jZ1A7qNq7,Purdue Boilermakers Womens Basketball vs. Ohio...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2025-12-31 17:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104
482,Z7r9jZ1A7qv_v,Purdue Boilermakers Mens Basketball vs. Washin...,Sports,Basketball,College,Mackey Arena,West Lafayette,IN,United States Of America,2026-01-07 00:00:00+00:00,...,40.4259,-86.9081,14876.0,1.0,1.0,1.0,1,1,0.362081,0.968104



✨ SIMILAR MATCHES (Top 10):


,event_id,name,segment,genre,subgenre,venue_name,city,state,country,datetime_utc,...,event_lat,event_lon,capacity,distance_score,genre_score,subgenre_score,segment_score,date_score,capacity_score,final_score
273,vv17fZbYGkMCVKy1,Indiana Pacers vs. Memphis Grizzlies,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-03-01 22:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
258,vv17fZbYGkMCSNzX,Indiana Pacers vs. Charlotte Hornets,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-02-27 00:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
19,vv17fZbYGkMCIKwY,Indiana Pacers vs. New York Knicks,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2025-12-19 00:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
51,vv17fZbYGkMCJSs4,Indiana Pacers vs. Sacramento Kings,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2025-12-09 00:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
20,vv17fZbYGkMCtNS6,Indiana Pacers vs. Boston Celtics,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2025-12-27 00:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
268,vv1AAZkGyGkd7SfnE,IHSAA Girls Basketball State Finals Session #2...,Sports,Basketball,High School,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-02-28 23:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
269,vv1AAZkGyGkd7w7n-,IHSAA Girls Basketball State Finals Session #1...,Sports,Basketball,High School,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-02-28 15:30:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
25,vv17fZbYGkMCwdMX,Indiana Pacers vs. Dallas Mavericks,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-02-22 22:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
21,vv17fZbYGkMClSSU,Indiana Pacers vs. Cleveland Cavaliers,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-01-07 00:00:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
22,vv17fZbYGkMCcdVm,Indiana Pacers vs. Boston Celtics,Sports,Basketball,NBA,Gainbridge Fieldhouse,Indianapolis,IN,United States Of America,2026-01-13 00:30:00+00:00,...,39.7684,-86.1581,17923.0,0.514853,1.0,0.0,1,1,0.375568,0.810561
